In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/tmp/ipykernel_1151375/3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3"
import warnings
warnings.filterwarnings("ignore")


In [ ]:
import os
os.chdir('../')

from matplotlib import pyplot as plt
import pandas as pd
import math
import numpy as np
from tqdm.auto import tqdm
from utils.classification_utils import simple_clf
from utils.classification_utils import  FocalBCELoss, threshold_output

from upath import UPath as Path
from torch.nn.functional import one_hot
from ome_zarr.reader import Reader
from ome_zarr.io import parse_url
from aicsimageio import AICSImage, transforms
from tqdm import tqdm 
import torch
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import torch.utils.data as data
from sklearn.metrics import accuracy_score, f1_score, classification_report
from skimage import io
import torchvision
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from scipy.spatial.distance import pdist, cdist
from sklearn import decomposition
from seaborn import clustermap
from harmony import harmonize
from scipy.spatial.distance import squareform
import scanpy as sc
from scipy.sparse.linalg import eigs
from scipy.stats import ttest_ind, zscore, norm
from scipy.stats import pearsonr
from matplotlib import cm
from umap import UMAP
from random import choices
import cuml

cmap = cm.nipy_spectral
t = torchvision.transforms.ToTensor()

phases = [
    'Interphase', 
    'prophase', 
    'early prometaphase', 
    'prometaphase/metaphase', 
    'anaphase/telophase paired',
    'anaphase/telophase unpaired', 
]


In [ ]:
from sklearn.model_selection import train_test_split

sorted_trained_features, sorted_df = torch.load('WTC11_data/pretrained_features_and_df.pth')
train_indices, test_indices = train_test_split(range(len(sorted_trained_features)), 
                                         train_size=0.8)
train_indices = torch.load('WTC11_data/train_indices.pth')
test_indices = torch.load('WTC11_data/test_indices.pth')
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(sorted_trained_features)
scaled_sorted_trained_features = scaler.transform(sorted_trained_features)
scaled_sorted_trained_features = sorted_trained_features

target_column = 'cell_stage'
train_x_dino = scaled_sorted_trained_features[train_indices]
test_x_dino = scaled_sorted_trained_features[test_indices]
train_x_xgb = sorted_df.iloc[train_indices]
test_x_xgb = sorted_df.iloc[test_indices]
train_y = sorted_df[target_column].values[train_indices]
test_y = sorted_df[target_column].values[test_indices]

label2int = dict(list(zip(sorted(np.unique(train_y)), range(len(np.unique(train_y))))))
numerical_train_y = list(map(lambda x:label2int[x], train_y))
numerical_test_y = list(map(lambda x:label2int[x], test_y))


from random import choices
def balance_data(X_train, y_train):
    k = len(y_train)
    freq_per_class = y_train.mean(axis=0)
    balance_freqs_per_class = 1 / (freq_per_class)
    balance_freq_per_sample = y_train * balance_freqs_per_class
    balance_freq_per_sample = (
        balance_freq_per_sample.max(axis=1) / balance_freqs_per_class.max()
    )
    indices = choices(np.arange(k), weights=balance_freq_per_sample, k=k)
    return (X_train[indices], y_train[indices], indices)
from torch.nn.functional import one_hot
one_hot_train_y = one_hot(torch.Tensor(numerical_train_y).long())
balanced_train_x_dino, balanced_numerical_train_y, balanced_train_indices = balance_data(train_x_dino, np.array(one_hot_train_y))
one_hot_test_y = one_hot(torch.Tensor(numerical_test_y).long())
balanced_test_x_dino, balanced_numerical_test_y, balanced_test_indices = balance_data(test_x_dino, np.array(one_hot_test_y))

classifier = simple_clf(
    train_x_dino.shape[1],
    one_hot_train_y.shape[1],
    p=0.5
)

import torch.utils.data as data
class AutoBalancedPrecomputedFeatures(data.Dataset):
    def __init__(self, inputs, labels, balance, **kwargs):
        self.features = inputs.detach().cpu()
        self.labels = labels.detach().cpu()
        self.idx = []
        self.num_samples = len(self.features)
        if balance:
            self.parse_labels()
        self.balance = balance

    def parse_labels(self):
        stats = pd.DataFrame(
            data=[
                {"class": u, "freq": self.labels[:, u].sum().item()}
                for u in range(self.labels.shape[1])
            ]
        )
        self.stats = stats.sort_values(by="freq")
        N = int(self.stats.freq.mean())
        print("Sampling", N, "samples per class for", self.labels.shape[1], "classes")
        self.N = N * self.labels.shape[1]

    def __getitem__(self, index):
        class_id = list(range(self.labels.shape[1]))[index % self.labels.shape[1]]
        if self.balance:
            sample_idx = np.random.choice(np.where(self.labels[:, class_id])[0], 1)
        else:
            sample_idx = index
        return (
            self.features[sample_idx],
            self.labels[sample_idx],
        )

    def __len__(self):
        return len(self.features)

train_ds = AutoBalancedPrecomputedFeatures(torch.Tensor(train_x_dino), one_hot_train_y, balance=True)
valid_ds = AutoBalancedPrecomputedFeatures(torch.Tensor(test_x_dino), one_hot_test_y, balance=False)


train_sampler = torch.utils.data.RandomSampler(train_ds)
test_sampler = torch.utils.data.SequentialSampler(valid_ds)
train_dl = torch.utils.data.DataLoader(
    train_ds,
    batch_size=int(224),
    sampler=train_sampler,
    num_workers=20,
    pin_memory=True,
    drop_last=False,
)
test_dl = torch.utils.data.DataLoader(
    valid_ds,
    batch_size=int(224),
    sampler=test_sampler,
    num_workers=20,
    pin_memory=True,
    drop_last=False,
)

from torch.optim.lr_scheduler import OneCycleLR, CosineAnnealingLR, CyclicLR, ConstantLR
lr = 0.00005
lr = (
    float(lr) * (int(512) * 1) / 256.0
)  # linear scaling rule

wd = 0.04
epochs = 100
optim = torch.optim.AdamW(
    classifier.parameters(),
    lr=float(lr),
    betas=(0.9, 0.999),
    eps=1e-08,
    weight_decay=float(wd),
    amsgrad=False,
)
total_steps = int(epochs) * len(train_dl)
scheduler = CyclicLR(
    optim,
    float(lr) / 25,
    float(lr),
    int(total_steps / 8),
    cycle_momentum=False,
)
criterion = torch.nn.BCEWithLogitsLoss()

from tqdm.auto import tqdm

def to_np(x):
    if isinstance(x, np.ndarray):
        return x
    if isinstance(x, torch.Tensor):
        return np.array(x.to("cpu"))
    return np.array(x)

def get_f1_score(all_targets, all_predictions):
    targets = to_np(torch.vstack(all_targets).detach().cpu())
    outputs = threshold_output(
        torch.vstack(all_predictions).detach().cpu(), use_sigmoid=True
    )
    score = f1_score(
        targets.reshape(targets.shape[0], targets.shape[-1]),
        outputs.reshape(outputs.shape[0], outputs.shape[-1]),
        average=None,
        zero_division=0,
    )
    return score    

def test_model(classifier, dl, device, criterion):
    classifier = classifier.eval()
    with torch.no_grad():
        predictions, all_targets, losses, all_features = [], [], [], []
        batch_pbar = tqdm(
            enumerate(dl),
            total=len(dl),
            unit=" test batches",
            position=1,
            leave=False,
        )
        for index, (images, targets) in batch_pbar:
            images = images.to(device)
            targets = targets.to(device)
            features = images
            outputs = classifier(features)
            loss = criterion(
                outputs.reshape(targets.shape[0], 1, targets.shape[-1]),
                targets.float().reshape(targets.shape[0], 1, targets.shape[-1]),
            )
            losses.append(loss.item())
            predictions.extend(outputs)
            # all_features.extend(features)
            all_targets.extend(targets)
    score = get_f1_score(all_targets, predictions)
    batch_pbar.close()
    classifier = classifier.train()
    return np.mean(losses), score, all_targets, predictions

import math
device = "cuda" if torch.cuda.is_available() else "cpu"
classifier.to(device)
epochs_to_run = epochs
epoch_pbar = tqdm(
    range(epochs_to_run),
    total=epochs_to_run,
    unit=" epochs",
    position=0,
    leave=True,
)
test_scores = []
for epoch in epoch_pbar:
    classifier = classifier.train()
    train_predictions, train_targets, train_loss = [], [], []
    batch_pbar = tqdm(
        enumerate(train_dl),
        total=len(train_dl),
        unit=" batches",
        position=1,
        leave=False,
    )
    for index, (images, targets) in batch_pbar:
        iteration = len(train_dl) * epoch + index  # global training iteration
        images = images.to(device)
        targets = targets.to(device)
        features = images

        outputs = classifier(features)
        loss = criterion(
            outputs, targets.float().reshape(targets.shape[0], 1, targets.shape[-1])
        )

        if not math.isfinite(loss.item()):
            print("Loss is {}, stopping training".format(loss.item()), force=True)
            sys.exit(1)

        optim.zero_grad()
        loss.backward()
        optim.step()
        scheduler.step()
    test_scores.append(np.mean(test_model(classifier, test_dl, device, criterion)[1]))
    print(test_scores[-1])

plt.plot(test_scores)
train_loss, train_score,all_targets, predictions  = test_model(classifier, train_dl, device, criterion)
test_loss, test_score,all_targets, predictions = test_model(classifier, test_dl, device, criterion)
print(f'Average test score: {np.mean(test_score)}')
print(f'Test score: {(test_score)}')
print(f'Train score: {np.mean(train_score)}')    

try:
    all_targets = torch.stack(all_targets)
except:
    pass
all_targets = all_targets.detach().cpu()

try:
    predictions = torch.stack(predictions)
except:
    pass
predictions = predictions.detach().cpu()
torch.save((all_targets, predictions), 'pretrained_model.pth')

In [ ]:
torch.set_num_threads(1)

from sklearn.model_selection import train_test_split

sorted_trained_features, sorted_df = torch.load('WTC11_data/DINO_features_and_df.pth')
train_indices, test_indices = train_test_split(range(len(sorted_trained_features)), 
                                         train_size=0.8)
train_indices = torch.load('WTC11_data/train_indices.pth')
test_indices = torch.load('WTC11_data/test_indices.pth')
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(sorted_trained_features)
scaled_sorted_trained_features = scaler.transform(sorted_trained_features)
scaled_sorted_trained_features = sorted_trained_features

# target_column = 'Protein'
target_column = 'cell_stage'
train_x_dino = scaled_sorted_trained_features[train_indices]
test_x_dino = scaled_sorted_trained_features[test_indices]
train_x_xgb = sorted_df.iloc[train_indices]
test_x_xgb = sorted_df.iloc[test_indices]
train_y = sorted_df[target_column].values[train_indices]
test_y = sorted_df[target_column].values[test_indices]
# train_y = sorted_df.cell_stage.values[train_indices]
# test_y = sorted_df.cell_stage.values[test_indices]

label2int = dict(list(zip(sorted(np.unique(train_y)), range(len(np.unique(train_y))))))
numerical_train_y = list(map(lambda x:label2int[x], train_y))
numerical_test_y = list(map(lambda x:label2int[x], test_y))


from random import choices
def balance_data(X_train, y_train):
    k = len(y_train)
    freq_per_class = y_train.mean(axis=0)
    balance_freqs_per_class = 1 / (freq_per_class)
    balance_freq_per_sample = y_train * balance_freqs_per_class
    balance_freq_per_sample = (
        balance_freq_per_sample.max(axis=1) / balance_freqs_per_class.max()
    )
    indices = choices(np.arange(k), weights=balance_freq_per_sample, k=k)
    return (X_train[indices], y_train[indices], indices)
from torch.nn.functional import one_hot
one_hot_train_y = one_hot(torch.Tensor(numerical_train_y).long())
balanced_train_x_dino, balanced_numerical_train_y, balanced_train_indices = balance_data(train_x_dino, np.array(one_hot_train_y))
one_hot_test_y = one_hot(torch.Tensor(numerical_test_y).long())
balanced_test_x_dino, balanced_numerical_test_y, balanced_test_indices = balance_data(test_x_dino, np.array(one_hot_test_y))


classifier = simple_clf(
    train_x_dino.shape[1],
    one_hot_train_y.shape[1],
    p=0.5
)

import torch.utils.data as data
class AutoBalancedPrecomputedFeatures(data.Dataset):
    def __init__(self, inputs, labels, balance, **kwargs):
        self.features = inputs.detach().cpu()
        self.labels = labels.detach().cpu()
        self.idx = []
        self.num_samples = len(self.features)
        if balance:
            self.parse_labels()
        self.balance = balance

    def parse_labels(self):
        stats = pd.DataFrame(
            data=[
                {"class": u, "freq": self.labels[:, u].sum().item()}
                for u in range(self.labels.shape[1])
            ]
        )
        self.stats = stats.sort_values(by="freq")
        N = int(self.stats.freq.mean())
        print("Sampling", N, "samples per class for", self.labels.shape[1], "classes")
        self.N = N * self.labels.shape[1]

    def __getitem__(self, index):
        class_id = list(range(self.labels.shape[1]))[index % self.labels.shape[1]]
        if self.balance:
            sample_idx = np.random.choice(np.where(self.labels[:, class_id])[0], 1)
        else:
            sample_idx = index
        return (
            self.features[sample_idx],
            self.labels[sample_idx],
        )

    def __len__(self):
        return len(self.features)

train_ds = AutoBalancedPrecomputedFeatures(torch.Tensor(train_x_dino), one_hot_train_y, balance=True)
valid_ds = AutoBalancedPrecomputedFeatures(torch.Tensor(test_x_dino), one_hot_test_y, balance=False)


train_sampler = torch.utils.data.RandomSampler(train_ds)
test_sampler = torch.utils.data.SequentialSampler(valid_ds)
train_dl = torch.utils.data.DataLoader(
    train_ds,
    batch_size=int(224),
    sampler=train_sampler,
    num_workers=10,
    pin_memory=False,
    drop_last=False,
)
test_dl = torch.utils.data.DataLoader(
    valid_ds,
    batch_size=int(224),
    sampler=test_sampler,
    num_workers=10,
    pin_memory=False,
    drop_last=False,
)

from torch.optim.lr_scheduler import OneCycleLR, CosineAnnealingLR, CyclicLR, ConstantLR
lr = 0.00005
lr = (
    float(lr) * (int(512) * 1) / 256.0
)  # linear scaling rule

wd = 0.04
epochs = 100
optim = torch.optim.AdamW(
    classifier.parameters(),
    lr=float(lr),
    betas=(0.9, 0.999),
    eps=1e-08,
    weight_decay=float(wd),
    amsgrad=False,
)
total_steps = int(epochs) * len(train_dl)
scheduler = CyclicLR(
    optim,
    float(lr) / 25,
    float(lr),
    int(total_steps / 8),
    cycle_momentum=False,
)
criterion = torch.nn.BCEWithLogitsLoss()

from tqdm.auto import tqdm

def to_np(x):
    if isinstance(x, np.ndarray):
        return x
    if isinstance(x, torch.Tensor):
        return np.array(x.to("cpu"))
    return np.array(x)

def get_f1_score(all_targets, all_predictions):
    targets = to_np(torch.vstack(all_targets).detach().cpu())
    outputs = threshold_output(
        torch.vstack(all_predictions).detach().cpu(), use_sigmoid=True
    )
    score = f1_score(
        targets.reshape(targets.shape[0], targets.shape[-1]),
        outputs.reshape(outputs.shape[0], outputs.shape[-1]),
        average=None,
        zero_division=0,
    )
    return score    

def test_model(classifier, dl, device, criterion):
    classifier = classifier.eval()
    with torch.no_grad():
        predictions, all_targets, losses, all_features = [], [], [], []
        batch_pbar = tqdm(
            enumerate(dl),
            total=len(dl),
            unit=" test batches",
            position=1,
            leave=False,
        )
        for index, (images, targets) in batch_pbar:
            images = images.to(device)
            targets = targets.to(device)
            features = images
            outputs = classifier(features)
            loss = criterion(
                outputs.reshape(targets.shape[0], 1, targets.shape[-1]),
                targets.float().reshape(targets.shape[0], 1, targets.shape[-1]),
            )
            losses.append(loss.item())
            predictions.extend(outputs)
            # all_features.extend(features)
            all_targets.extend(targets)
    score = get_f1_score(all_targets, predictions)
    batch_pbar.close()
    classifier = classifier.train()
    return np.mean(losses), score, all_targets, predictions

import math
device = "cuda" if torch.cuda.is_available() else "cpu"
classifier.to(device)
epochs_to_run = epochs
epoch_pbar = tqdm(
    range(epochs_to_run),
    total=epochs_to_run,
    unit=" epochs",
    position=0,
    leave=True,
)
test_scores = []
for epoch in epoch_pbar:
    classifier = classifier.train()
    train_predictions, train_targets, train_loss = [], [], []
    batch_pbar = tqdm(
        enumerate(train_dl),
        total=len(train_dl),
        unit=" batches",
        position=1,
        leave=False,
    )
    for index, (images, targets) in batch_pbar:
        iteration = len(train_dl) * epoch + index  # global training iteration
        images = images.to(device)
        targets = targets.to(device)
        features = images

        outputs = classifier(features)
        loss = criterion(
            outputs, targets.float().reshape(targets.shape[0], 1, targets.shape[-1])
        )

        if not math.isfinite(loss.item()):
            print("Loss is {}, stopping training".format(loss.item()), force=True)
            sys.exit(1)

        optim.zero_grad()
        loss.backward()
        optim.step()
        scheduler.step()
    test_scores.append(np.mean(test_model(classifier, test_dl, device, criterion)[1]))
    print(test_scores[-1])

plt.plot(test_scores)
train_loss, train_score,all_targets, predictions  = test_model(classifier, train_dl, device, criterion)
test_loss, test_score,all_targets, predictions = test_model(classifier, test_dl, device, criterion)
print(f'Average test score: {np.mean(test_score)}')
print(f'Test score: {(test_score)}')
print(f'Train score: {np.mean(train_score)}')    

try:
    all_targets = torch.stack(all_targets)
except:
    pass
all_targets = all_targets.detach().cpu()

try:
    predictions = torch.stack(predictions)
except:
    pass
predictions = predictions.detach().cpu()
torch.save((all_targets, predictions), 'trained_model.pth')

In [ ]:
sorted_df = pd.read_csv('WTC11_data/normalized_cell_df.csv')
xgb_feature_df = sorted_df[['angle', 'bbox_max_x', 'bbox_max_y', 'bbox_max_z',
       'bbox_min_x', 'bbox_min_y', 'bbox_min_z', 'bf_clip_hi', 'bf_clip_lo',
       'cell_height', 'cell_surface_area', 'cell_volume',
        'dna_clip_hi', 'dna_clip_lo', 'membrane_clip_hi', 'membrane_clip_lo', 
        'nuclear_height', 'nuclear_surface_area',
       'nuclear_volume', 'shape_mode_1_height', 'shape_mode_2_volume',
       'shape_mode_3_major_tilt', 'shape_mode_4_minor_tilt',
       'shape_mode_5_elongation', 'shape_mode_6_bean-ness',
       'shape_mode_7_pear-ness', 'shape_mode_8_wedge', 'meta_colony_centroid_1', 'meta_colony_centroid_2']]
engineered_values = xgb_feature_df.values
engineered_values, values = torch.load('WTC11_data/engineered_features.pth')

target_column = 'Protein'
target_column = 'cell_stage'

train_indices = torch.load('WTC11_data/train_indices.pth')
test_indices = torch.load('WTC11_data/test_indices.pth')

train_x_xgb = engineered_values[train_indices]
test_x_xgb = engineered_values[test_indices]
train_y = sorted_df[target_column].values[train_indices]
test_y = sorted_df[target_column].values[test_indices]
label2int = dict(list(zip(sorted(np.unique(train_y)), range(len(np.unique(train_y))))))
numerical_train_y = list(map(lambda x:label2int[x], train_y))
numerical_test_y = list(map(lambda x:label2int[x], test_y))
one_hot_train_y = one_hot(torch.Tensor(numerical_train_y).long())
one_hot_test_y = one_hot(torch.Tensor(numerical_test_y).long())


In [ ]:
one_hot_train_y = one_hot(torch.Tensor(numerical_train_y).long())
balanced_train_x_xgboost, balanced_numerical_train_y, balanced_train_indices = balance_data(train_x_xgb, np.array(one_hot_train_y))
one_hot_test_y = one_hot(torch.Tensor(numerical_test_y).long())
balanced_test_x_xgboost, balanced_numerical_test_y, balanced_test_indices = balance_data(test_x_xgb, np.array(one_hot_test_y))
balanced_numerical_train_y = np.array(numerical_train_y)[balanced_train_indices]
balanced_numerical_test_y = np.array(numerical_test_y)[balanced_test_indices]


In [ ]:
import xgboost as xgb
model = XGBClassifier()
model.fit(balanced_train_x_xgboost, np.array(balanced_numerical_train_y))



In [ ]:
y_pred = model.predict(test_x_xgb)
predictions = [round(value) for value in y_pred]
xgbscores = f1_score(numerical_test_y, predictions, average=None)
print(xgbscores)
print(np.mean(xgbscores))
torch.save((numerical_test_y, predictions), 'xgb_model.pth')
